In [2]:
# load data
import pickle
import numpy as np

with open("piano_X.pkl", "rb") as f:
    X = pickle.load(f)
    print('X:',len(X))

with open("piano_y.pkl", "rb") as f:
    y = pickle.load(f)
    print('X:',len(X))

X: 673098
X: 673098


In [3]:
import pandas as pd
y = pd.DataFrame(y, columns=['pitch','velocity'])
y = y['pitch'].astype(int)

In [4]:
y

0         57
1         81
2         62
3         86
4         64
          ..
673093    64
673094    67
673095    72
673096    40
673097    43
Name: pitch, Length: 673098, dtype: int32

In [5]:
# scale and split

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
for i, note in enumerate(X):
    X[i] = scaler.fit_transform(note) # transform the whole thing



In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # split

In [9]:
type(X_train)

numpy.ndarray

In [50]:
# CNN classification model design

# import tensorflow as tf
# from tensorflow.keras import layers, models
# from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# # Define the CNN model
# model = models.Sequential()

# # Input layer
# model.add(layers.Input(shape=(13, 50)))  # Assuming 50 time steps and 13 coefficients

# # Convolutional layers
# model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'))
# model.add(layers.MaxPooling1D(pool_size=2))
# model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'))
# model.add(layers.MaxPooling1D(pool_size=2))
# model.add(layers.Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'))
# model.add(layers.MaxPooling1D(pool_size=2))

# # Flatten layer
# model.add(layers.Flatten())

# # Fully connected layers
# model.add(layers.Dense(units=256, activation='relu'))
# model.add(layers.Dropout(0.5))
# model.add(layers.Dense(units=128, activation='relu'))

# # Output layer
# num_pitch_classes = 87  # Adjust based on your pitch classes
# model.add(layers.Dense(units=num_pitch_classes, activation='softmax'))

# # Compile the model
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Summary of the model architecture
# model.summary()

In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Input
from tensorflow.keras.models import Model


input_shape = (13, 50)

model = tf.keras.Sequential()
model.add(Input(shape=input_shape))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='linear'))  # Linear activation for regression
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 11, 64)              │           9,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 5, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 320)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │          41,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 50,881 (198.75 KB)

 Trainable params: 50,881 (198.75 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
model = tf.keras.Sequential()
model.add(Input(shape=input_shape))
model.add(tf.keras.layers.LSTM(64, activation='relu', return_sequences=True))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='linear'))  # Linear activation for regression
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 13, 64)              │          29,440 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 832)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │         106,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 136,193 (532.00 KB)

 Trainable params: 136,193 (532.00 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
results = model.fit(X_train, y_train, 
                    epochs=15, batch_size=512,
                    validation_data=(X_test, y_test))

Epoch 1/15
1052/1052 ━━━━━━━━━━━━━━━━━━━━ 17s 15ms/step - loss: 538.0358 - mae: 15.7148 - val_loss: 155.3843 - val_mae: 10.1360
Epoch 2/15
1052/1052 ━━━━━━━━━━━━━━━━━━━━ 15s 15ms/step - loss: 147.0864 - mae: 9.6721 - val_loss: 143.8731 - val_mae: 9.4714
Epoch 3/15
1052/1052 ━━━━━━━━━━━━━━━━━━━━ 15s 14ms/step - loss: 143.6192 - mae: 9.5460 - val_loss: 138.0002 - val_mae: 9.3183
Epoch 4/15
1052/1052 ━━━━━━━━━━━━━━━━━━━━ 15s 15ms/step - loss: 138.6965 - mae: 9.3353 - val_loss: 134.9695 - val_mae: 9.1992
Epoch 5/15
1052/1052 ━━━━━━━━━━━━━━━━━━━━ 15s 15ms/step - loss: 134.9279 - mae: 9.1856 - val_loss: 131.7814 - val_mae: 9.0377
Epoch 6/15
1052/1052 ━━━━━━━━━━━━━━━━━━━━ 15s 14ms/step - loss: 131.9217 - mae: 9.0467 - val_loss: 131.1471 - val_mae: 9.0011
Epoch 7/15
1052/1052 ━━━━━━━━━━━━━━━━━━━━ 15s 14ms/step - loss: 130.8624 - mae: 9.0000 - val_loss: 131.2252 - val_mae: 8.8961
Epoch 8/15
1052/1052 ━━━━━━━━━━━━━━━━━━━━ 15s 15ms/step - loss: 129.7215 - mae: 8.9388 - val_loss: 127.9544 - val_ma

In [11]:
model.save('piano_mfccs_RNN-LSTM.keras')

NameError: name 'X_train' is not defined

In [ ]:
# I keep getting mae close to 8. This is not ideal, even considered horrible, but that surprisingly is a whole octave